# Oil Price Prediction

This notebook loads a cleaned dataset and applies:
- Classic Machine Learning models (Linear Regression, Random Forest, etc.)
- Time Series forecasting using Prophet
- Rerun EVERY cells when you change the resample_frequency to get different time granularity

to predict the future WTI Crude oil prices.

In [ ]:
# Cell 1: Import Libraries and Utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings

# Import the utility classes
from oil_prediction_utils import *
warnings.filterwarnings('ignore')
print("All libraries and utilities imported successfully!")


In [ ]:
file_path = '/content/relevant_features_1.csv'
df = pd.read_csv(file_path)
print("Dataset loaded successfully!")

In [ ]:
resample_frequency = 'W'  # Change to 'D', 'W', or 'M'

# Get configuration using the utility
config = DataProcessor.get_frequency_config(resample_frequency)
freq_str = config['freq_str']
lags = config['lags']
window_sizes = config['window_sizes']
initial_train_periods = config['initial_train_periods']
test_periods = config['test_periods']
forecast_periods = config['forecast_periods']
seasonal_period = config['seasonal_period']

print(f"Configuration set for {freq_str}ly frequency.")
print(f"   - Lags: {lags}")
print(f"   - Windows: {window_sizes}")
print(f"   - Test Periods: {test_periods}")
print(f"   - Forecast Periods: {forecast_periods}")
print(f"   - SARIMA Seasonal Period: {seasonal_period}")

In [ ]:
# Cell 3: Data Preprocessing
df_processed = DataProcessor.preprocess_data(df, resample_frequency)
print(f"Data preprocessed and resampled to {freq_str}ly frequency.")
df_processed.head()

In [ ]:
# Cell 4: Initialize Components
target_col = 'WTI_Crude'
date_col = 'Date'

# Initialize the backtester
backtester = ModelBacktester(df_processed, target_col, date_col)
print("Backtester initialized successfully!")


In [ ]:
# Cell 5 & 6: Run Backtesting
print(f"Starting backtesting with {test_periods} test periods...")

backtest_results = backtester.walk_forward_validation_realistic(
    initial_train_periods=initial_train_periods,
    total_test_periods=test_periods,
    p_lags=lags,
    p_window_sizes=window_sizes,
    seasonal_period=seasonal_period
)

print(f"Calculating performance metrics for all models ({freq_str}ly)...")
performance_metrics = backtester.calculate_metrics()

print(f"REALISTIC BACKTESTING PERFORMANCE METRICS ({freq_str.upper()}LY)")
print("="*70)
print(performance_metrics.round(4))
print("="*70)


In [ ]:
# Cell 7: Future Forecasting Setup
forecaster = ModelForecaster(target_col, date_col)

# Prepare full dataset for final training
print("Preparing full dataset features for final model training...")
full_featured_data = backtester.prepare_features_historical_only(
    df_processed, lags, window_sizes
).dropna()

feature_cols = backtester.get_feature_columns(full_featured_data)
X_full = full_featured_data[feature_cols]
y_full = full_featured_data[target_col]

print(f"Training data prepared up to: {full_featured_data[date_col].max().strftime('%Y-%m-%d')}")

In [ ]:
# Cell 8: Generate Future Forecasts for All Models
future_forecasts_all_models = {}

# Models to forecast (excluding ARIMA and SARIMA)
models_to_forecast = ['Linear Regression', 'Ridge Regression', 'Random Forest',
                     'Gradient Boosting', 'Prophet']

for model_name in models_to_forecast:
    print(f"Generating future forecast for: {model_name}")

    try:
        if model_name == 'Prophet':
            # Prophet forecasting
            future_predictions_df = forecaster.forecast_prophet_model(
                full_featured_data, feature_cols, forecast_periods, resample_frequency
            )

        else:
            # Sklearn models forecasting
            sklearn_models = forecaster.get_sklearn_models()
            final_model = sklearn_models[model_name]
            final_model.fit(X_full, y_full)

            # Get recent data for iterative forecasting
            max_feature_period = max(max(lags), max(window_sizes))
            last_known_data = df_processed.iloc[-max_feature_period:].copy()

            future_predictions_df = forecaster.forecast_sklearn_model(
                final_model, full_featured_data, last_known_data,
                forecast_periods, resample_frequency, lags, window_sizes
            )

        future_forecasts_all_models[model_name] = future_predictions_df
        print(f"Forecast generated successfully for {model_name}")

    except Exception as e:
        print(f"Error generating forecast for {model_name}: {e}")
        future_forecasts_all_models[model_name] = pd.DataFrame({date_col: [], 'Forecast': []})

print("Future forecasts generated for all selected models.")

In [ ]:
# Cell 9: Structure Data for Power BI Export
print("Structuring data for Power BI export...")

combined_data_export = DataExporter.structure_data_for_powerbi(
    df_processed, backtest_results, future_forecasts_all_models,
    target_col, date_col, exclude_models=['ARIMA', 'SARIMA']
)

print("Data structured successfully!")
print("Sample of combined data:")
print(combined_data_export.head())
print(combined_data_export.tail())

## Structure data for power bi (revised)

### Subtask:
Combine historical actuals, backtest predictions for all models, and future forecasts for all models into a single DataFrame with 'Date', 'DataType', 'Model', and 'Value' columns.


In [ ]:
# Cell 10: Export to CSV
combined_data_filename = f'combined_oil_price_data_{resample_frequency}.csv'
DataExporter.export_to_csv(combined_data_export, combined_data_filename)

# Also export performance metrics
metrics_filename = f'performance_metrics_{resample_frequency}.csv'
performance_metrics.to_csv(metrics_filename, index=True)
print(f"Performance metrics exported to '{metrics_filename}'")


In [ ]:
# Cell 11: Visualization
print("Creating comprehensive visualization...")

# Determine forecast start date
forecast_start_date = df_processed[date_col].max()

# Create the plot
Visualizer.plot_results(
    combined_data_export,
    target_col='Value',  # Note: Value column in combined data
    date_col=date_col,
    forecast_start_date=forecast_start_date,
    title=f'{target_col} Price: Historical, Backtest & Forecast ({freq_str}ly Data)'
)

## Export to csv (revised)

### Subtask:
Save the combined DataFrame to a CSV file.


In [ ]:
# Cell 12: Best Model Analysis and Individual Forecast
if not performance_metrics.empty and 'R²' in performance_metrics.columns:
    # Find best model based on R²
    best_model_name = performance_metrics.dropna()['R²'].idxmax()
    print(f"Best model based on R²: {best_model_name}")
    print(f"Performance: {performance_metrics.loc[best_model_name].round(4)}")

    # Plot individual best model forecast
    if best_model_name in future_forecasts_all_models:
        best_forecast = future_forecasts_all_models[best_model_name]

        plt.figure(figsize=(16, 8))

        # Plot recent historical data
        recent_periods = forecast_periods * 3
        recent_data = df_processed.iloc[-recent_periods:]
        plt.plot(recent_data[date_col], recent_data[target_col],
                label=f'Historical {target_col}', color='royalblue', linewidth=2)

        # Plot best model forecast
        if not best_forecast.empty:
            plt.plot(best_forecast[date_col], best_forecast['Forecast'],
                    label=f'{best_model_name} Forecast', color='darkorange',
                    linestyle='--', marker='o', markersize=4)

        plt.axvline(forecast_start_date, color='red', linestyle=':',
                   linewidth=2, label='Forecast Start')

        plt.title(f'{target_col}: Best Model ({best_model_name}) Forecast',
                 fontsize=16, fontweight='bold')
        plt.xlabel('Date', fontsize=12)
        plt.ylabel(f'Price (USD)', fontsize=12)
        plt.legend(fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()


In [ ]:
# Cell 13: Summary Statistics
print("\n" + "="*60)
print("ANALYSIS SUMMARY")
print("="*60)

print(f"Data Frequency: {freq_str}ly")
print(f"Historical Data Points: {len(df_processed)}")
print(f"Backtest Periods: {len(backtest_results) if 'backtest_results' in locals() else 'N/A'}")
print(f"Forecast Periods: {forecast_periods}")

print(f"\nModels Evaluated: {len(performance_metrics)}")
for model in performance_metrics.index:
    r2 = performance_metrics.loc[model, 'R²']
    mae = performance_metrics.loc[model, 'MAE']
    dir_acc = performance_metrics.loc[model, 'Directional_Accuracy']
    print(f"  {model}: R²={r2:.3f}, MAE=${mae:.2f}, Dir.Acc={dir_acc:.1f}%")

print(f"\nData Export:")
print(f"  Combined Data: {combined_data_filename}")
print(f"  Performance Metrics: {metrics_filename}")
print(f"  Total Records in Combined Data: {len(combined_data_export)}")

data_type_counts = combined_data_export['DataType'].value_counts()
print(f"\nData Type Distribution:")
for dtype, count in data_type_counts.items():
    print(f"  {dtype}: {count} records")

print("\nPower BI Integration:")
print("The exported CSV contains 'Date', 'DataType', 'Model', and 'Value' columns.")
print("Use 'DataType' and 'Model' as slicers in Power BI for interactive analysis.")
print("Create time series visualizations filtered by data type and model.")


## Summary:

### Data Analysis Key Findings

*   Future forecasts were successfully generated for Linear Regression, Ridge Regression, Random Forest, Gradient Boosting, and Prophet models for the specified number of periods.
*   The final DataFrame, containing the 'Date', 'DataType', 'Model', and 'Value' columns, was exported to a CSV file named `combined_oil_price_data_{time}.csv`.

### Insights or Next Steps

*   The structured CSV file with 'DataType' and 'Model' columns is ideal for creating interactive dashboards in Power BI, allowing users to easily compare actuals, backtests, and forecasts across different models using slicers.
*   The exported CSV file can be directly imported into Power BI for visualization and further analysis, facilitating the creation of time-series plots filtered by data type and model.


In [ ]:
# Cell 14: Validation Check
print("\n" + "="*50)
print("DATA VALIDATION")
print("="*50)

# Check for missing values
missing_values = combined_data_export.isnull().sum()
print(f"Missing Values Check:")
for col, missing in missing_values.items():
    if missing > 0:
        print(f"  {col}: {missing} missing values")
    else:
        print(f"  {col}: No missing values")

# Check date ranges
print(f"\nDate Range Validation:")
print(f"  Earliest Date: {combined_data_export[date_col].min()}")
print(f"  Latest Date: {combined_data_export[date_col].max()}")
print(f"  Total Date Range: {(combined_data_export[date_col].max() - combined_data_export[date_col].min()).days} days")

# Check value ranges
print(f"\nValue Range Validation:")
print(f"  Minimum Value: ${combined_data_export['Value'].min():.2f}")
print(f"  Maximum Value: ${combined_data_export['Value'].max():.2f}")
print(f"  Mean Value: ${combined_data_export['Value'].mean():.2f}")

print("\nValidation complete - data is ready for Power BI import!")